# Dubins 2D Aircraft Rejoin Example

This Jupyter notebook demonstrates a multi-agent task scenario with control system analysis framework (CSAF), where a group of Dubins aircraft attempt to rejoin in formation and collectively fly at a specific heading angle. Dubins aircraft presents a dynamically simple 2D aircraft model, taken from the [AerospaceRL repository on GitHub](https://github.com/act3-ace/aerospaceRL). The state space is 3D, being two position coordinates $(x,y)$
and a heading angle $\theta$. The control action simply is to apply a heading angular rate $\dot \theta$ while maintaining constant velocity
(no throttle). The update equation is

 \begin{equation}
  \dot{\mathbf x} = \begin{bmatrix}
v \cos (x_2) \\
v \sin (x_2) \\
u \\
\end{bmatrix},
\end{equation}

where $\mathbf{x} = (x, y, \theta)$, and $v$ is some fixed airspeed parameter.


## Controller Design

A lateral rejoin task is specified: given **n** planes at different orientations, produce **n** maneuver sequences that allow them to be no further
than some terminal length apart $r_l$ and at some terminal heading angle $\theta_t$. Given that the only control surface that can be affected is angular
rate, the following control scheme is formulated,

1.  Associate each plane with neighbors that should be considered in collision avoidance; construct a graph $\mathcal G = (\mathcal P, \mathcal C)$ with vertices of planes $\mathcal P$ and edges of neighbors $\mathcal C$. In this case, a simple $k$-neighbors was done,
    with $k=1$; solve only for the nearest neighbor. This method can be extended to different graph constructions, by
    appropriately weighting the graph edges inversely to the distances between nodes via a weighted adjacency matrix.
2.  Solve for the angle that will cause a plane $s_i$ and its nearest neighbor $s_j$ to approach one another the fastest,
     \begin{equation}
     \theta_{i}(s_i, s_j) = \operatorname{atan2}(x_{j1} -x_{i1}, x_{j0} - x_{i0}).
     \end{equation}
3.  Linearly combine the angle $\theta_i$ and the terminal heading angle $\theta_t$.
    Apply some weight that is a function of the distance between the aircraft $r$, $w: \overline{\mathbb R^-}  \rightarrow [0, 1]$,
    \begin{equation}
    \theta_c(s_i) = w(r(s_i, s_j)) \theta_t + (1-w(r(s_i, s_j))) \theta_j.
    \end{equation}
    In this example,
    \begin{equation}
    w(r) = \exp\left( -\frac{(r-r_l)^2}{\tau} \right),
    \end{equation}
    where $r_l$ is the desired final distance between aircraft, and $\tau$ is a hyperparameter characterizing how soon to apply the collision avoidance correction.
4.  This solves for the desired heading angle of the aircraft. As the heading angular rate is the input, control the position quantity via a proportional controller,
    \begin{equation}
    u = k_p (\theta_c - \theta).
    \end{equation}
    
    <div>
<img src="img/dubins_rejoin_diag.png" width="500"/>
</div>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Configuration

To model the multi-agent scenario with CSAF, each aircraft agent is modeled as a single component. Each agent is parameterized with the following component configuration:

In [ ]:
!cat /csaf-system/components/dubinplant.toml

Each agent is assigned an identity index `idx` in addition to the parameters mentioned in the linear model; the controller uses this index to assign each agent its angular rate. This index value, as well as the initial state, is changed inside the configuration object before system creation. 

In [ ]:
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/rejoin_config.toml")

In [ ]:
def set_dub_idx(conf, identifier, number):
    conf.config_dict['components'][identifier+str(number)]\
    ['config']['parameters']['idx'] = number
    
def set_dub_state(conf, identifier, number, state):
    my_conf.config_dict['components'][identifier+str(number)]\
    ['config']['topics']['states']['initial'] = state

In [ ]:
states = [[0, 0, np.deg2rad(45)],
              [-5, -10, np.deg2rad(-30)],
            [-3, -15, np.deg2rad(90)],
            [0, -20, np.deg2rad(0)]]

for idx, state in enumerate(states):
    set_dub_idx(my_conf, 'jet', idx)
    set_dub_state(my_conf, 'jet', idx, state)

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

## Simulation

After configuration, the CSAF system is simulated via the method `System.simulate_tspan`. The returned traces `trajs` capture the trajectories of each agent as the controller applies the angular rate.

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 25.0]

# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

In [ ]:
states = [np.array(trajs[f"jet{idx}"].states) for idx in range(4)]

In [ ]:
# show aircraft trajectories
cs = ['g', 'r', 'b', 'k']
fig, ax = plt.subplots(figsize=(15, 7))
for idx in range(4):
    plt.plot(*states[idx][:, :2].T*10, c=cs[idx])
plt.title("Aircraft Trajectories")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.show()

## Animation

When running the Jupyter notebook, animation is available to view the trajectories as a movie. Uncomment the following code and run.

In [ ]:
#%matplotlib notebook
#import sys
#sys.path.append('/csaf-system')
#from rejoin_plot import plot_air

#ani = plot_air(states)
#from IPython.display import HTML
#HTML(ani.to_jshtml())